<a href="https://colab.research.google.com/github/SS1031/kaggle-shopee/blob/main/notebooks/001_create_tfrecode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipython-autotime

In [ ]:
%load_ext autotime

time: 96.9 µs (started: 2021-04-04 14:44:40 +00:00)


In [ ]:
!nvidia-smi

Sun Apr  4 14:44:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!mkdir -p "/kaggle/input/"
!cp "/content/drive/MyDrive/kaggle-shopee/shopee-product-matching.zip" "/kaggle/input/"

time: 1min 2s (started: 2021-04-04 14:45:57 +00:00)


In [ ]:
!unzip "/kaggle/input/shopee-product-matching.zip" -d "/kaggle/input/shopee-product-matching"
!rm -rf "/kaggle/input/shopee-product-matching.zip

In [ ]:
import gc
import glob
import os

import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

print(f"Numpy version : {np.__version__}")
print(f"Pandas version : {pd.__version__}")
print(f"Tensorflow version : {tf.__version__}")
print(f"Pillow version : {PIL.__version__}")


Numpy version : 1.19.5
Pandas version : 1.1.5
Tensorflow version : 2.4.1
Pillow version : 7.1.2
time: 578 ms (started: 2021-04-04 15:27:09 +00:00)


In [ ]:

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()  # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def _list_float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))


def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _list_int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


time: 9.58 ms (started: 2021-04-04 15:27:12 +00:00)


In [ ]:
df_train = pd.read_csv("/kaggle/input/shopee-product-matching/train.csv")
df_train["enc_label_group"] = LabelEncoder().fit_transform(df_train["label_group"])

time: 104 ms (started: 2021-04-04 15:27:14 +00:00)


In [ ]:

RESIZE_WIDTH = 512
RESIZE_HEIGHT = 512

TFRECORD_MAX_SIZE = 80 * 1024 * 1024 # 80 MB

TOTAL_IMAGES = len(df_train.index)
# TOTAL_IMAGES = len(df_test.index)

# part 1 : 0:TOTAL_IMAGES // 2 (train) [CURRENT]
# part 2 : TOTAL_IMAGES // 2:TOTAL_IMAGES (train)
# part 3 : 0:TOTAL_IMAGES (test)
START_INDEX = 0
END_INDEX = TOTAL_IMAGES // 2

BATCH_IMAGE = 1024

SEED = 42

time: 3.96 ms (started: 2021-04-04 15:32:13 +00:00)


In [ ]:
def create_tfrecord(fold, df, dir="shopee-skf-tf-records"):
    fold = str(fold).zfill(2)
    if not os.path.exists(dir):
        os.makedirs(dir)
    curr_file = f"{dir}/train-skf-fold{fold}-{len(df)}.tfrec"
    print(f"Writing to {curr_file}")
    writer = tf.io.TFRecordWriter(curr_file)
    for _, row in df.iterrows():
        image = f"/kaggle/input/shopee-product-matching/train_images/{row['image']}"
        img = open(image, "rb")
        img_read = img.read()
        image_decoded = tf.image.decode_jpeg(img_read, channels=3)
        resized_img = tf.image.resize_with_pad(
            image_decoded,
            target_width=RESIZE_WIDTH,
            target_height=RESIZE_HEIGHT,
            method=tf.image.ResizeMethod.BILINEAR,
        )
        resized_img = tf.cast(resized_img, tf.uint8)
        resized_img = tf.io.encode_jpeg(resized_img)

        feature = {
            "posting_id": _bytes_feature(tf.compat.as_bytes(row["posting_id"])),
            "image": _bytes_feature(resized_img),
            "image_phash": _bytes_feature(tf.compat.as_bytes(row["image_phash"])),
            "title": _bytes_feature(tf.compat.as_bytes(row["title"])),
            "label_group": _int64_feature(row["label_group"]),
            "enc_label_group": _int64_feature(row["enc_label_group"]),
        }
        example = tf.train.Example(features=tf.train.Features(feature=feature))
        writer.write(example.SerializeToString())
    writer.close()


time: 20.6 ms (started: 2021-04-04 15:43:15 +00:00)


In [ ]:
skf = StratifiedKFold(n_splits=12, random_state=SEED, shuffle=True)
for fold, (_, idx) in enumerate(skf.split(df_train, df_train["enc_label_group"])):
    create_tfrecord(fold, df_train.iloc[idx, :])
    

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=12.
  % (min_groups, self.n_splits)), UserWarning)


Writing to shopee-skf-tf-records/train-skf-fold00-2855.tfrec
Writing to shopee-skf-tf-records/train-skf-fold01-2855.tfrec
Writing to shopee-skf-tf-records/train-skf-fold02-2854.tfrec
Writing to shopee-skf-tf-records/train-skf-fold03-2854.tfrec
Writing to shopee-skf-tf-records/train-skf-fold04-2854.tfrec
Writing to shopee-skf-tf-records/train-skf-fold05-2854.tfrec
